In [295]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## A brief introduction to Algorithmic Music Composition (in Python!)

### First, let's import some modules.

In [296]:
import numpy as np # NumPy allows us to do math with arrays.  
import cosmos # a custom library for working with audio

In [297]:
# load an audio sample
snare = cosmos.load_audio("snare.wav")
# and listen to it
cosmos.play_audio(snare)


In [298]:
# create an empty audio track
track = cosmos.AudioTrack(duration=5)
# add the snare sample to it
track.add_audio(snare, time=0.0) # at time 0 (in seconds)
track.add_audio(snare, time=1.0) # and at time 1
cosmos.play_audio(track)

In [299]:
# clear the audio track
track.clear()
cosmos.play_audio(track)

In [300]:
track.clear()
# add the snare sample at specified times
times = [0, 0.25, 0.75, 1.25, 1.5, 1.625, 2, 2.125]
# use a for loop to add the snare sample at each time
for time in times:
    track.add_audio(snare, time=time)

cosmos.play_audio(track)

In [301]:
track.clear()
# set the gain of the snare sample (ranging from 0 to 1)
# so that it gets quieter over time
track.add_audio(snare, time=0.0, gain=1.0)
track.add_audio(snare, time=0.25, gain=0.5)
track.add_audio(snare, time=0.5, gain=0.25)
track.add_audio(snare, time=0.75, gain=0.125)
track.add_audio(snare, time=1.0, gain=0.0625)
track.add_audio(snare, time=1.25, gain=0.03125)
track.add_audio(snare, time=1.5, gain=0.015625)
track.add_audio(snare, time=1.625, gain=0.0078125)
cosmos.play_audio(track)

In [302]:
# Same thing, but with more efficient code
track.clear()
# set the array of times using a list comprehension
# this is equivalent to building the array using a for loop:
''' 
times = []
 for i in range(8):
    times.append(i/4)
'''
times = [i/4 for i in range(8)] # [0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75]

# set the array of gains using a list comprehension
# this is equivalent to building the array using a for loop:
'''
gains = []
for i in range(8):
    gains.append(2 ** -i)
'''
gains = [2 ** -i for i in range(8)] # [1.0, 0.5, 0.25, 0.125, 0.0625, 0.03125, 0.015625, 0.0078125]

for i in range(len(times)):
    track.add_audio(snare, time=times[i], gain=gains[i])
cosmos.play_audio(track)

In [303]:
track.clear()
# faster, longer, slower decay
times = [i/14 for i in range(32)]
gains = [2 ** (-0.25 * i) for i in range(32)]

print([round(i, 2) for i in gains])
for i in range(len(times)):
    track.add_audio(snare, time=times[i], gain=gains[i])
cosmos.play_audio(track)

[1.0, 0.84, 0.71, 0.59, 0.5, 0.42, 0.35, 0.3, 0.25, 0.21, 0.18, 0.15, 0.12, 0.11, 0.09, 0.07, 0.06, 0.05, 0.04, 0.04, 0.03, 0.03, 0.02, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0]


## Patterns

In [304]:
# Create a pattern of times (using durations) and gains
track = cosmos.AudioTrack(duration=20)

durations = [1/2, 1/4, 1/2, 1/4, 1/8, 3/8]
# get the times by cumulatively summing the durations
# times = [sum(durations[:i]) for i in range(len(durations))] 
gains = [0.3, 0.7, 0.3, 1, 0.7, 0.3]
pattern = cosmos.Pattern(durations=durations, gains=gains, samples=snare, cycles=4)
track.add_audio(pattern.to_audio())
# pattern.add_to_track(track, start_time=0.0, cycles=4)
cosmos.play_audio(track)


In [307]:
# vary the samples
track.clear()

hihat = cosmos.load_audio("hihat.wav")
samples = [hihat, hihat, snare, hihat, hihat, snare]
pattern = cosmos.Pattern(durations=durations, gains=gains, samples=samples, cycles=2)
track.add_audio(pattern.to_audio(), time=0.0)
next_time = pattern.dur_tot
pattern.set_cycle_duration(pattern.cycle_dur / 2)
pattern.cycles *= 2 # double the number of cycles
track.add_audio(pattern.to_audio(), time=next_time)
cosmos.play_audio(track)


In [38]:
duration = 60
sample_rate = 48000
durs = np.random.uniform(0, 1, 200)
durs /= np.sum(durs)
durs *= duration
print(np.sum(durs))
onsets = np.cumsum(durs)
onsets -= onsets[0]

# onsets = np.random.uniform(0, duration, 100)
score = np.zeros((sample_rate * duration, 2))
for onset in onsets:
    start = int(onset * sample_rate)
    end = start + snare.shape[0]
    score[start:end, :] += snare

# score[0: snare.shape[0], :] += snare
print(score.shape[0]/sample_rate)

60.0
60.0
